# Sales Outreach Agent

In [30]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from openai.types.responses import ResponseTextDeltaEvent

# Load environment variables
load_dotenv(override=True)

True

In [31]:
# Define agent personas with distinct tones

professional_prompt = (
    "You are a professional sales agent at TechLink, "
    "an online education company offering AI-driven, career-focused learning programs. "
    "Your goal is to write serious, persuasive cold outreach emails targeted at professionals and educational institutions."
)

engaging_prompt = (
    "You are a witty, engaging sales agent at TechLink, "
    "an online education company offering AI-powered, upskilling programs for tech and business roles. "
    "Craft humorous, attention-grabbing cold emails that spark curiosity and encourage responses."
)

concise_prompt = (
    "You are a fast-moving, efficient sales agent at TechLink, "
    "an online education platform offering smart, AI-powered learning paths. "
    "Write short, compelling cold emails that quickly highlight value and drive action."
)

# Create three distinct AI agents
professional_agent = Agent(
    name="Professional Sales Agent",
    instructions=professional_prompt,
    model="gpt-4o-mini"
)

engaging_agent = Agent(
    name="Engaging Sales Agent",
    instructions=engaging_prompt,
    model="gpt-4o-mini"
)

concise_agent = Agent(
    name="Concise Sales Agent",
    instructions=concise_prompt,
    model="gpt-4o-mini"
)


In [16]:
# Stream and display email content from an agent
email_result = Runner.run_streamed(engaging_agent, input="Write a cold sales email")
async for event in email_result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Unlock Your Future Faster Than a Cat on a Roomba! 🐱🚀

Hi [First Name],

Ever watched a cat ride a Roomba? It’s mesmerizing, and believe me, that’s exactly how fast your career could be zooming forward with our AI-powered upskilling programs at TechLink!

Think about it: in the time it takes to fill your coffee cup, you could be acquiring skills that’ll have employers wondering if you’re a wizard—except instead of casting spells, you’ll be magic with data and tech!

Why TechLink, you ask? 
- **AI-Powered Learning**: Our programs learn how you learn, so it’s like having a personal trainer for your brain!
- **Flexible Schedules**: Learn at your own pace—coffee breaks included, of course!
- **Real-World Projects**: Tackle projects that make your resume pop like popcorn in the microwave!

Curious to see how we can turn your career trajectory from “meh” to “WOW”? Let’s chat! I promise, it’ll be more fun than a cat in a box!

Looking forward to your reply (preferably before I crave a

In [17]:
import asyncio

# The email topic or objective
email_topic = "Write a cold sales email"

# Run all sales agents concurrently to generate email variations
agent_outputs = await asyncio.gather(
    Runner.run(professional_agent,email_topic),
    Runner.run(engaging_agent,email_topic),
    Runner.run(concise_agent,email_topic),
)

# Extract the final text output from each agent
cold_emails = [response.final_output for response in agent_outputs]

# Display all generated emails
for idx, email in enumerate(cold_emails, start=1):
    print(f"\n--- Email Option {idx} ---\n{email}\n")


--- Email Option 1 ---
Subject: Elevate Your Team’s Skills with AI-Driven Learning

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I represent TechLink, an innovative online education company dedicated to empowering professionals through AI-driven, career-focused learning programs.

In today’s rapidly evolving job market, staying ahead means continuously upgrading skills and knowledge. At TechLink, we offer tailored learning solutions that not only equip your team with the latest industry-relevant skills but also empower them to thrive in an increasingly competitive landscape.

Our programs feature:

- **Personalized Learning Paths**: Adaptive courses designed to meet individual career goals.
- **Real-World Applications**: Hands-on projects and case studies to ensure applicable skills.
- **Expert-Led Instruction**: Insights from industry leaders who share relevant experience and wisdom.

We’ve successfully partnered with organizations like [e

In [20]:
# Create an evaluator agent to choose the most appealing email
selector_prompt = (
    "You are a prospective customer receiving cold outreach emails. "
    "Choose the email you're most likely to respond to, based solely on content. "
    "Don't explain your choice—just return the best email as-is."
)

email_selector_agent = Agent(
    name="Email Selector Agent",
    instructions=selector_prompt,
    model="gpt-4o-mini"
)

# Evaluate all email options and pick the most compelling one
with trace("Evaluate & select best cold email"):
    agent_outputs = await asyncio.gather(
        Runner.run(professional_agent,email_topic),
        Runner.run(engaging_agent,email_topic),
        Runner.run(concise_agent,email_topic),
    )

    # Extract email outputs
    email_variants = [res.final_output for res in agent_outputs]

    # Combine all into a single prompt for the selector agent
    evaluation_prompt = "Evaluate the following cold sales emails:\n\n" + "\n\n---\n\n".join(email_variants)

    # Run selector agent
    selected_response = await Runner.run(email_selector_agent, evaluation_prompt)
    
    print("\nBest Sales Email Selected:\n")
    print(selected_response.final_output)




Best Sales Email Selected:

Subject: Transform Your Team's Skills with AI-Driven Learning

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I’m reaching out from TechLink, where we specialize in revolutionizing professional development through our AI-driven, career-focused learning programs.

In today’s fast-evolving landscape, staying ahead of the curve is essential. We understand that your team’s growth is a priority and that continuous learning is crucial for maintaining a competitive edge. That’s where TechLink can make a significant impact.

Our programs are tailored to meet the unique needs of professionals and educational institutions, offering:

- **Personalized Learning Paths:** Our AI algorithms assess individual skill levels and recommend targeted courses, ensuring your team engages with the most relevant content.
- **Career-Focused Curriculum:** Each program is designed to align with industry demands, equipping learners with the ski

Now go and check out the trace:

https://platform.openai.com/traces

## Use Agents and Function as Tool

In [21]:
import sendgrid
import os
from sendgrid.helpers.mail import Email, To, Content, Mail
from agents import function_tool, Agent, trace

# Define a tool function to send an email
@function_tool
def send_sales_email(email_body: str):
     """Send the provided email body to the sales prospect."""
     sg = sendgrid.SendGridAPIClient(api_key=os.environ.get("SENDGRID_API_KEY"))
     sender = Email("anshulc55@gmail.com")
     recipient = To("anshulc55@icloud.com")
     content = Content("text/plain", email_body)
     email = Mail(sender, recipient, "Cold Outreach from TechLink", content).get()
     response = sg.client.mail.send.post(request_body=email)
     return {"status": "sent"}

In [22]:
send_sales_email

FunctionTool(name='send_sales_email', description='Send the provided email body to the sales prospect.', params_json_schema={'properties': {'email_body': {'title': 'Email Body', 'type': 'string'}}, 'required': ['email_body'], 'title': 'send_sales_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x11ac63d80>, strict_json_schema=True)

### Convert Agents into Tools

In [23]:
# Wrap agents as callable tools

tool_pro = professional_agent.as_tool(
    tool_name="professional_email_generator",
    tool_description="Generates a serious, professional cold email."
)

tool_fun = engaging_agent.as_tool(
    tool_name="witty_email_generator",
    tool_description="Creates a humorous and engaging sales email."
)

tool_brief = concise_agent.as_tool(
    tool_name="brief_email_generator",
    tool_description="Produces a short, direct sales email."
)

# Define the full toolset available to the manager agent
toolset = [tool_pro, tool_fun, tool_brief, send_sales_email]


## Define the Sales Manager Agent with tool-based instructions

In [24]:
# Sales Manager only uses tools — no email writing on its own
manager_prompt = (
    "You are a sales manager at TechLink, responsible for outbound outreach. "
    "You are provided with several email-writing tools, each with a unique style. "
    "Your task is to try all three once, evaluate which email is the best for the user, "
    "and then use the 'send_sales_email' tool to send only the best email. "
    "Do NOT generate your own email — only use tools."
)

sales_manager_agent = Agent(
    name="Sales Manager",
    instructions=manager_prompt,
    tools=toolset,
    model="gpt-4o-mini"
)

# Use the Sales Manager to run the end-to-end orchestration
message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales Manager Workflow"):
    final_result = await Runner.run(sales_manager_agent, message)

Now go and check out the trace:

https://platform.openai.com/traces

### ⚠️ Wait — You Didn't Get an Email?

> 🙏 Thanks to student **La Hui L.** for describing this issue and sharing helpful fixes.

If your AI agent ran successfully, but you didn't receive an email, follow these steps to troubleshoot:

---

#### 🔍 Step-by-Step Troubleshooting:

1. **Check Your Spam Folder**
   - Many students found their emails in **Spam** or **Promotions** folders.
   - Mark the email as "Not Spam" to ensure future delivery.

2. **Print the Result Object**
   - Run the following in your notebook:
     ```python
     print(result)
     ```
   - Look closely for any error messages, especially those related to **SSL** or **network issues**.

3. **If You See SSL Errors**
   - You may need to adjust networking settings on your local machine.
   - Check out this helpful guide for common networking issues:  
     🔗 [Networking Tips & Fixes](https://chatgpt.com/share/680620ec-3b30-8012-8c26-ca86693d0e3d)

4. **Review Your OpenAI Trace**
   - OpenAI’s dashboard provides trace logs — use them to inspect tool calls and failures.
   - Look for `"tool_call"` or `"tool_result"` in the timeline for clues.

5. **Check SendGrid Activity Logs**
   - Visit [SendGrid Dashboard → Email Activity](https://app.sendgrid.com/email_activity)
   - Look for bounce messages, failures, or blocks.
   - Make sure your **sender email is verified**.

---

🧠 **Still Need Help?**

- Post your error message in the course discussion board.
- Share the output of `print(result)` or a screenshot (no credentials).
- Your instructor is here to help!



### 💡 Extra Tip for Windows Users – SSL Certificate Error Fix

If you're on **Windows** and encounter **SSL certificate errors** when sending emails using SendGrid, try this:

---

#### 🛠 Step-by-Step Fix:

1. **Upgrade Certificate Bundle**  
   Open your terminal and run:

   ```bash
   uv pip install --upgrade certifi


In [ ]:
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()

# HandOff Workflows

In [32]:
from agents import Agent

# Subject Line Generator
subject_agent = Agent(
    name="SubjectLineWriter",
    instructions="Generate a subject line for a cold email that improves open rates.",
    model="gpt-4o-mini"
)

# HTML Converter Agent
html_agent = Agent(
    name="HTMLFormatter",
    instructions="Convert plain text emails into well-structured, visually clear HTML format.",
    model="gpt-4o-mini"
)

subject_tool = subject_agent.as_tool(
    tool_name="generate_subject", 
    tool_description="Generate a cold email subject line."
    )

html_tool = html_agent.as_tool(
    tool_name="convert_to_html", 
    tool_description="Convert text email to HTML."
    )

# Function Tool to Send Email via SendGrid
@function_tool
def send_email(subject: str, html_body: str):
     """Send an HTML email with subject and body to the recipient list."""
     sg = sendgrid.SendGridAPIClient(api_key=os.environ.get("SENDGRID_API_KEY"))
     sender = Email("anshulc55@gmail.com")
     recipient = To("anshulc55@icloud.com")
     content = Content("text/html", html_body)
     email = Mail(sender, recipient, subject, content).get()
     response = sg.client.mail.send.post(request_body=email)
     return {"status": "sent"}

# Combine all tools for formatting and sending
email_formatter_tools = [subject_tool, html_tool, send_email]
email_formatter_tools

[FunctionTool(name='generate_subject', description='Generate a cold email subject line.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'generate_subject_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x1198cb1a0>, strict_json_schema=True),
 FunctionTool(name='convert_to_html', description='Convert text email to HTML.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'convert_to_html_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x11b8f9f80>, strict_json_schema=True),
 FunctionTool(name='send_email', description='Send an HTML email with subject and body to the recipient list.', params_json_schema={'properties': {'subject': {'title': 'Subje

### Create a Specialized Email Manager Agent (Handoff Target)

In [33]:
email_manager_prompt = (
        "You are responsible for taking a raw email body and sending it out. "
        "First, use the subject generator tool. Then, convert the email to HTML format. "
        "Finally, use the send_email tool to deliver the message."
    )

# HandOff Agent
email_manager_agent = Agent(
    name="EmailManager",
    instructions=email_manager_prompt,
    tools=email_formatter_tools,
    model="gpt-4o-mini",
    handoff_description="Format and send an email using subject generation and HTML conversion."
)


# These are your content generators
email_generator_tools = [tool_pro, tool_fun, tool_brief]

### Sales Manager Agent Using Both Tools & Handoff

In [ ]:
# The Sales Manager Agent coordinates everything
sales_manager_prompt = (
        "You are a sales manager tasked with sending the best possible cold email from your company TechLink. "
        "Use the available email-writing tools to generate several options. "
        "Choose the most persuasive email. Then, hand off to the EmailManager agent to format and send the email."
    )

sales_manager_agent = Agent(
    name="SalesManager",
    instructions=sales_manager_prompt,
    tools=email_generator_tools,
    handoffs=[email_manager_agent],
    model="gpt-4o-mini"
)

# Run the Full Workflow
prompt = "Send out a cold sales email addressed to Dear CEO from Anshul."
with trace("Automated Sales Email Flow"):
    result = await Runner.run(sales_manager_agent, prompt)


### Remember to check the trace

https://platform.openai.com/traces

And then check your email!!

## 📝 Note: Google’s Agent Development Kit (ADK)

Google has recently announced their own **Agent Development Kit (ADK)** — currently in **early preview**.

While it’s too soon to include ADK in this course (as it's still evolving), it's interesting to note how **similar its structure and concepts are** to OpenAI's Agents SDK.

Here’s a quick peek at a sample ADK snippet from Google:

```python
root_agent = Agent(
    name="weather_time_agent",
    model="gemini-2.0-flash",
    description="Agent to answer questions about the time and weather in a city.",
    instruction="You are a helpful agent who can answer user questions about the time and weather in a city.",
    tools=[get_weather, get_current_time]
)
